# Algorithms 1. Week 4. Five algorithms.
### Bohdan Tymofieienko, B. S. 
#### btymofieienko@student.fontys.nl
##### 13 March 2022

## 1. Main algorithm. Shortest path

I chose shortest path problem because it has application in many fields. The task required to design and implement an algorithm for finding the shortest path from source vertex to all the others. This is often called as "single-source shortest path". One of the most famous algorithm for this case is Dijkstra algorithm.

Dijkstra algorithm:

1. Assign source distances to $1$.<br>
2. Assign all the distances to $\infty$.<br>
3. For all the vertices that are adjacent to current pointer:<br>
    3.1 Relax the edge to the vertex if path is shorter.<br>
4. Find not visited vertex with the minimum distance and set it as current.<br>

### 1.1 Dijkstra. Research.

Dijkstra is a greedy algorithm because it makes a *locally** optimal decision *(see step 4)* with the hope it will reach globally optimal solution at some point. However, as you might observe it doesn't use backtracking simply because there is no need in it. Import aspect is so called "relaxation". This is in essence the foundation and main operation in the algorithm. Relaxation is the process of a method that repeatedly decreases an upper bound on the actual shortest path weight of each vertex until the upper bound equivalent the shortest - path weigh *[1]*. It is important to remember that Dijkstra algorithm works only on non-negative weighted graphs.

### 1.2 Dijkstra. Implementation.

First of all we need to design a graph data structure to model the graph itself. For this I defined two classes - edge and graph. Graph contains a list of edges and number of vertices as fields. It has one public method - add. This is done for convenience. However it also has two other private methods, one of them assures that graph stays connected and the other one prevents adding an edge with negative weight. As it was mentioned before these are important conditions for Dijkstra algorithm. 

In [1]:
class Edge:
  def __init__(self, origin, dest, weight):
    self.origin = origin
    self.dest = dest
    self.weight = weight

In [2]:
class Graph:
  def __init__(self):
    self.edges = []
    self.verticesNumber = 0
    
  def __isConnected(self, origin, dest):
    if len(self.edges) != 0:
      for e in self.edges:
        if e.origin == origin or e.dest == dest or e.origin == dest or e.dest == origin:
          return True
      return False
    else: return True
  
  def __isVertexNew(self, vertex):
    for e in self.edges:
        if e.origin == vertex or e.dest == vertex:
          return False
    return True
  
  def add(self,origin, dest, weight):
    if weight > 0 and self.__isConnected(origin, dest):      
      if self.__isVertexNew(origin):
        self.verticesNumber+=1
      if self.__isVertexNew(dest):
        self.verticesNumber+=1
      self.edges.append(Edge(origin, dest, weight))
      

There exist multiple implementations of Dijkstra. Most advance of them use priority queue to improve time complexity. I learned about that after designing the algorithm so you don't see this in my solution. However I do take this in consideration in analysis.

In [3]:
def findMin(table, visited):
  min = max(table, key = table.get)
  for v in table:
    if table[v] < table[min] and v not in visited:
      min = v
  return min   

Going further, the algorithm it self. You may observe that I keep a table, which is essentially a list of pairs of the type (vertex, distance to vertex) and update it when needed. I use a dictionary comprehension to reduce amount of code. 

In [4]:
def Dijkstra(graph, origin):
  visited = []
  table = {i:float('inf') for i in range(graph.verticesNumber)}
  table[origin] = 0
  current = origin
  #repeat untill all vertices are visited
  while len(visited) != graph.verticesNumber:
    visited.append(current)
    #find all possible edges from origin
    for e in graph.edges:
      if e.origin == current:
        #if new path is shorter. Relaxation
        if table[e.origin] + e.weight < table[e.dest]: 
          #update the table
          table[e.dest] = table[e.origin] + e.weight
    current = findMin(table, visited)
  return table

### 1.3 Dijkstra. Analysis.

To compute the time complexity I will need to analyze the loop sequence. First of all we can see while loop. It will take exactly $O(|V|)$ to execute, where $|V|$ stands for amount of vertices. Then another loop comes in place. It is obvious that it runs $O(|E|)$, where $|E|$ stands for amount of edges. Additionally, *findMin()* function goes through all the elements in table so that's $O(|V|)$. This results in complexity for inner part of $O(|V| + |E|)$. Lastly we combine inner and outer part and the result complexity is $O(|V| \times (|V| + |E|))$.

#### 1.3.1 Dijkstra. Potential improvement.

As I mentioned above there is an improvement potential for this implementation. In particular, using a priority queue data structure instead of using *findMin()* function has a positive effect on time complexity. It can get better up to $O(log(|V|) \times (|V| + |E|))$ given we are using priority queue implemented with a Fibonacci heap.

### 1.4 Dijkstra. Testing.

I have attached the diagram with solved shortest path problem, and you can of course compare the results below. 
<div>
<img src="Screenshot%202022-03-14%20110335.png" width="500"/>
</div>

In [5]:
def isEqual(result, actual):
  return result == actual

In [6]:
myGraph = Graph()

myGraph.add(0, 1, 1)
myGraph.add(1, 0, 1)

myGraph.add(1, 3, 1)
myGraph.add(3, 1, 1)

myGraph.add(0, 2, 6)
myGraph.add(2, 0, 6)

myGraph.add(2, 1, 2)
myGraph.add(1, 2, 2)

myGraph.add(2, 3, 2)
myGraph.add(3, 2, 2)

myGraph.add(2, 4, 5)
myGraph.add(4, 2, 5)

myGraph.add(3, 4, 5)
myGraph.add(4, 3, 5)

print(Dijkstra(myGraph, 0))

{0: 0, 1: 1, 2: 3, 3: 2, 4: 7}


In [7]:
isEqual(Dijkstra(myGraph, 0), {0: 0, 1: 1, 2: 3, 3: 2, 4: 7})

True

**Time measurement** 

In [ ]:
%%timeit
Dijkstra(myGraph, 0)

## 2. The way out. Maze

First of all I would model the maze in form two dimensional array. Since there are separators in form of "1" symbols (or walls) we can think of this matrix as of a graph. To find the shortest path it is enough to find the minimum amounts of "0" you follow to get to an end ("2"). There are multiple ways to approach it. However I propose to use a slightly modified BFS algorithm to find a solution. Algorithm consists of two parts - BFS and finding an actual sequence of steps to the end.

This part propagate values decremented by 1 up to the end.
Algorithm (Part 1):

1. Assign START coordinates to pointer.<br>
2. While value of a pointer is not 2 repeat:<br>
    2.1 Decrement the value of a pointer by 1 <br>
    2.2 Add all the pointer's neighbors which are "0" to the queue.<br>
    2.3 Pop the first element from a queue and assign it to the pointer.<br>
    
Algorithm pseudocode (Part 1):
1. pointer = start.<br>
2. while not valueOf(pointer) == 2:<br>
    2.1 valueOf(pointer) = valueOf(pointer) - 1<br>
    2.2 forall neighbors n : if n == 0 then queue.push(n) <br>
    2.3 pointer = queue.pop()<br>
    
This part backtracks the shortest path by the use of modified matrix from part 1.    
Algorithm (Part 2):  
1. Loop through neighbors of the END to find the minimum one.<br>
2. Assign it coordinates to pointer.<br>   
3. While value of a pointer is not -1 repeat:<br>
   3.1. Add it to the front of path array.<br>
   3.2. Loop through neighbors of the pointer to find value less by one from pointer's value.<br>
   3.3. Assign it to pointer.<br>
   
Algorithm pseudocode (Part 2):
1. pointer = END <br>
2. path = []
3. while not valueOf(pointer) == -1:<br>
    3.1 path.addToFront(pointer)
    3.2 neighbors = []
    3.3 forall n : if adjacent(n,pointer) then neighbors.add(n)<br>
    3.4 pointer = min(getNeighbours(END))<br>
4. return path    
    
It is important to decrement and not increment to not coincide with 1 (wall) or 2 (end). The magnitude, however, is not affected by that. For example path of -9 can be interpreted as path of 9.

<div>
<img src="maze.jpg" width="500"/>
</div>

## 3. Cryptarythmetic puzzle

For this one there might exist multiple ways to solve it. Of course basic brute force with many for loops is possible. However it is extremely inefficient so we won't even consider it. Relatively optimal, however, is backtracking approach. The basic idea could be to have a set of possible characters and obviously the mechanism to verify if the arrangement worked. First of all we will need to place them consecutively as they are in the set. In the case arrangement didn't work we shall try unassign last n elements and try other. The algorithm could look like following: 

1. Assign not used value from the set consecutively with regard to some set of conditions* <br>
2. If arrangement worked then return arrangement<br>
3. Else unassign last n elements and call the function again.<br>
4. If there are no combinations left return false.

'*' - Of course there exist multiple optimizations that will speed up the solution. For example left most character cannot be 0. I can also think of something like a carry. By making all these observations it is possible to make a certain set of constraints that will later on "cut of" impossible branches from decision space and save time.

## 4. From X to Y

To solve this problem correctly we have to consider all the possible outcomes of operations we apply. Only then we can conclude that amount of steps is minimal. Unless there might exist some sort of a formula for these three particular operation, but we will not take that in consideration since it defeats the purpose of the problem. So my approach would be to start at X and try to apply all possible operations. After that however we have to keep applying the rule until we get to our result. In this way we construct a tree of possible operations and operations on their outcomes. This can be done with the BFS algorithm. As soon as we reach the result we can stop, this means we found the minimal sequence. (Keep going won't make any sense because amount of steps will increase). Also, we can be really sure that we found a minimal amount of steps since we literally tried all the possible cases. One thing can be to keep track of resulting numbers in order to avoid problem of having loops, since tree can't have loops.

1. Create a root. (X value) <br>
2. Enqueue the root and type of operation. <br>
3. While queue is not empty: <br>
    3.1 Dequeue the last element  <br>
    3.2 If element is equal to Y then break <br>
    3.3 Apply all the operations. <br>
    3.4 Enqueue the results. <br>
4. Backtrack the sequence and return it. <br>


## 5. Longest Common Subsequence 

This problem can be solved in a very primitive way by generating all the possible subsets of both string and then comparing them. However, this is absolutely inefficient. Dynamic programming can help in solving it appropriately. In particular, we can solve break it down up to trivial problem by assuming three possible scenarios. Scenario one states that both strings end with the same character. From that we can make an important conclusion, this means that now the LCS of both strings is LCS of reduced by 1 strings + last common character. We can recursively repeat that until we get both string have length 0, however there might occur the situation when they don't end with the same character. However, in this case we can still benefit because we now for sure that LCS does not end with one of these so we can reject another one.

Algorithm:
Consider there are two strings A and B with lengths m and n respectively, then

LCS(A,B):
1. If m is 0 or n is 0 then return <br>
2. Else if A[m] is equal to B[n] then return LCS(A[m-1], B[n-1]) <br>
3. Else return maximum(LCS(A[m-1],B[n]),LCS(A[m],B[n-1]))<br>
    


## 6. Literature.

[1] Relaxation. https://www.javatpoint.com/relaxation<br>